In [1]:
import sys
import os
sys.path.append('../')

from tqdm import tqdm

import pandas as pd
from PIL import Image, ImageOps
from tqdm import tqdm
from glob import glob

#Base on https://github.com/aosokin/os2d github and annotation

In [ ]:
# !wget 'https://drive.usercontent.google.com/download?id=1Fx9lvmjthe3aOqjvKc6MJpMuLF22I1Hp&export=download&authuser=1&confirm=t' -O grozi.zip 
# !wget 'https://drive.usercontent.google.com/download?id=1Vp8sm9zBOdshYvND9EPuYIu0O9Yo346J&export=download&authuser=1&confirm=t' -O retail_test_sets.zip
!unzip retail_test_sets.zip
!unzip grozi.zip

In [2]:
paths = ['images',
         'images/train',
         'images/test',
         'images/valid',
         'images/dairy',
         'images/paste',
         'labels',
         'labels/train',
         'labels/test',
         'labels/valid',
         'labels/dairy',
         'labels/paste',
         'supports',
         ]

for path in paths:
    if not os.path.exists(path):
        os.mkdir(path)


In [3]:
p1 = pd.read_csv("dairy/classes/dairy.csv")
p1.head()


,gtbboxid,classid,imageid,lx,rx,ty,by,difficult
0,899,0,0,0.0007,0.0169,0.1784,0.4371,1
1,900,0,0,0.0189,0.0980,0.1733,0.4345,0
2,428,1,4,0.7875,0.8250,0.1098,0.2586,1
3,429,1,4,0.8213,0.8712,0.1113,0.2602,0
4,430,1,0,0.0885,0.1601,0.1853,0.4500,0


In [4]:
p2 = pd.read_csv("paste/classes/paste.csv")
p2.head()

,gtbboxid,classid,imageid,lx,rx,ty,by,difficult
0,0,3543,16454,0.0351,0.0785,0.0324,0.1354,0
1,1,3543,16454,0.0728,0.1178,0.0336,0.1362,0
2,2,3543,16454,0.1105,0.1508,0.0309,0.1346,0
3,3,3552,16454,0.1451,0.1921,0.0324,0.1381,0
4,4,3552,16454,0.1854,0.2335,0.0336,0.1373,0


In [5]:
p3 = pd.read_csv("grozi/classes/grozi.csv")
p3.head()

,gtbboxid,classid,imageid,lx,rx,ty,by,difficult,split
0,0,0,0,0.1075,0.2350,0.0000,0.2433,0,train
1,1,0,0,0.2350,0.3775,0.0000,0.2500,0,train
2,2,1,0,0.3900,0.5250,0.0033,0.2700,0,train
3,3,1,0,0.5200,0.6500,0.0200,0.2700,0,train
4,4,1,0,0.6450,0.7650,0.0467,0.2733,0,train


In [10]:
def resize_with_padding(img, expected_size):
    width, height = img.size
    ratio = width / height
    if width < height:
        img = img.resize((int(ratio*expected_size[0]),int(expected_size[0])))
    else:
        img = img.resize((int(expected_size[1]),int((1/ratio)*expected_size[1])))
        
    img.thumbnail((expected_size[0], expected_size[1]))
    delta_width = expected_size[0] - img.size[0]
    delta_height = expected_size[1] - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(img, padding, (114,114,114))

In [11]:
def resize_label_with_padding(old_size, expected_size, bbox):
    (x,y,w,h) = bbox

    ratio = old_size[0] / old_size[1]
    if old_size[0] < old_size[1]:
        new = (int(ratio*expected_size[0]),int(expected_size[0]))
    else:
        new = (int(expected_size[1]),int((1/ratio)*expected_size[1]))

    delta_width = expected_size[0] - new[0]
    delta_height = expected_size[1] - new[1]

    pad_width = delta_width // 2
    pad_height = delta_height // 2

    x_t = (pad_width +  (x)*new[0]) / expected_size[0]
    y_t = (pad_height + (y)*new[1]) / expected_size[1]
    w_t = w*new[0] / expected_size[0]
    h_t = h*new[1] / expected_size[1]
    
    return x_t,y_t,w_t,h_t

In [12]:
changement = {}
i = 0

# support
for classid in tqdm(p3['classid'].unique()):
    image = Image.open("grozi/classes/images/"+str(classid)+".jpg")
    image = resize_with_padding(image,(160,160))
    changement[classid] = i
    if not os.path.exists('supports/'+str(i)):
        os.mkdir('supports/'+str(i))    
    image.save("supports/"+str(changement[classid])+'/0.png')
    i = i + 1

100%|██████████| 1063/1063 [00:04<00:00, 258.67it/s]


In [13]:
d = p3.loc[p3['split'] == 'train']

y = 0

for imageid in tqdm(d['imageid'].unique()):
    path_img = "images/train/"
    path_lab = "labels/train/"
    image = Image.open("grozi/src/3264/"+str(imageid)+".jpg")
    width,height = image.size
    image = resize_with_padding(image,(640,640)) #max
    image.save(path_img+str(y)+'.png')
    m = d.loc[d['imageid'] == imageid]
    with open(path_lab+str(y)+'.txt', 'wt') as f:
        # [box_left, box_top, box_right, box_bottom], 1) # lx ty rx by
        for c, lx, rx, ty, by in m[['classid','lx',	'rx','ty','by']].iloc(): 
            cx = (lx + rx) /2
            cy = (ty + by) / 2
            w = rx - lx
            h = by - ty

            cx,cy,w,h = resize_label_with_padding((width,height), (640,640), (cx,cy,w,h)) #max
            s = str(changement[c])+' '+str(cx)+' '+str(cy)+' '+str(w)+' '+str(h)+' \n'
            f.write(s)
    y = y + 1

100%|██████████| 596/596 [01:13<00:00,  8.10it/s]


In [14]:
d = p3.loc[p3['split'] == 'val-old-cl']

y = 0

for imageid in tqdm(d['imageid'].unique()):
    path_img = "images/valid/"
    path_lab = "labels/valid/" 
    image = Image.open("grozi/src/3264/"+str(imageid)+".jpg")
    width,height = image.size
    image = resize_with_padding(image,(640,640)) #max
    image.save(path_img+str(y)+'.png')
    m = d.loc[d['imageid'] == imageid]
    with open(path_lab+str(y)+'.txt', 'wt') as f:
        for c,lx,rx,ty,by in m[['classid','lx',	'rx',	'ty',	'by']].iloc(): # [box_left, box_top, box_right, box_bottom], 1) # lx ty rx by
            cx = (rx + lx) /2
            cy = (by + ty) / 2
            w = rx - lx
            h = by - ty
            cx,cy,w,h = resize_label_with_padding((width,height), (640,640), (cx,cy,w,h)) #max
            s = str(changement[c])+' '+str(cx)+' '+str(cy)+' '+str(w)+' '+str(h)+' \n'
            f.write(s)
    y = y + 1

100%|██████████| 60/60 [00:07<00:00,  8.09it/s]


In [15]:
d = p3.loc[p3['split'] == 'val-new-cl']

y = 0

for imageid in tqdm(d['imageid'].unique()):
    path_img = "images/test/"
    path_lab = "labels/test/" 
    image = Image.open("grozi/src/3264/"+str(imageid)+".jpg")
    width,height = image.size
    image = resize_with_padding(image,(640,640)) #max
    image.save(path_img+str(y)+'.png')
    m = d.loc[d['imageid'] == imageid]
    with open(path_lab+str(y)+'.txt', 'wt') as f:
        for c,lx,rx,ty,by in m[['classid','lx',	'rx',	'ty',	'by']].iloc(): # [box_left, box_top, box_right, box_bottom], 1) # lx ty rx by
            cx = (rx + lx) /2
            cy = (by + ty) / 2
            w = rx - lx
            h = by - ty
            cx,cy,w,h = resize_label_with_padding((width,height), (640,640), (cx,cy,w,h)) #max
            s = str(changement[c])+' '+str(cx)+' '+str(cy)+' '+str(w)+' '+str(h)+' \n'
            f.write(s)
    y = y + 1

100%|██████████| 84/84 [00:10<00:00,  8.16it/s]


In [16]:
# Dairy datasets add
changement = {}

for classid in tqdm(p1['classid'].unique()):
    image = Image.open("dairy/classes/images/"+str(classid)+".jpg")
    image = resize_with_padding(image,(160,160))
    changement[classid] = i
    if not os.path.exists('supports/'+str(i)):
        os.mkdir('supports/'+str(i))  
    image.save("supports/"+str(changement[classid])+'/0.png')
    i = i + 1

d = p1

y = 0

for imageid in tqdm(d['imageid'].unique()):
    path_img = "images/dairy/"
    path_lab = "labels/dairy/" 
    image = Image.open("dairy/src/original/"+str(imageid)+".jpg")
    width,height = image.size
    image = resize_with_padding(image,(640,640)) #max
    image.save(path_img+str(y)+'.png')
    m = d.loc[d['imageid'] == imageid]
    with open(path_lab+str(y)+'.txt', 'wt') as f:
        for c,lx,rx,ty,by in m[['classid','lx',	'rx',	'ty',	'by']].iloc(): # [box_left, box_top, box_right, box_bottom], 1) # lx ty rx by
            cx = (rx + lx) /2
            cy = (by + ty) / 2
            w = rx - lx
            h = by - ty
            cx,cy,w,h = resize_label_with_padding((width,height), (640,640), (cx,cy,w,h)) #max
            s = str(changement[c])+' '+str(cx)+' '+str(cy)+' '+str(w)+' '+str(h)+' \n'
            f.write(s)
    y = y + 1

100%|██████████| 12/12 [00:01<00:00,  6.07it/s]


In [17]:
# For paste Datasets

changement = {}

for classid in tqdm(p2['classid'].unique()):
    image = Image.open("paste/classes/images/"+str(classid)+".jpg")
    image = resize_with_padding(image,(160,160))
    changement[classid] = i
    if not os.path.exists('supports/'+str(i)):
        os.mkdir('supports/'+str(i))  
    image.save("supports/"+str(changement[classid])+'/0.png')
    i = i + 1

d = p2

y = 0

for imageid in tqdm(d['imageid'].unique()):
    path_img = "images/paste/"
    path_lab = "labels/paste/" 
    image = Image.open("paste/src/original/"+str(imageid)+".jpg")
    width,height = image.size
    image = resize_with_padding(image,(640,640)) #max
    image.save(path_img+str(y)+'.png')
    m = d.loc[d['imageid'] == imageid]
    with open(path_lab+str(y)+'.txt', 'wt') as f:
        for c,lx,rx,ty,by in m[['classid','lx',	'rx',	'ty',	'by']].iloc(): # [box_left, box_top, box_right, box_bottom], 1) # lx ty rx by
            cx = (rx + lx) /2
            cy = (by + ty) / 2
            w = rx - lx
            h = by - ty
            cx,cy,w,h = resize_label_with_padding((width,height), (640,640), (cx,cy,w,h)) #max
            s = str(changement[c])+' '+str(cx)+' '+str(cy)+' '+str(w)+' '+str(h)+' \n'
            f.write(s)
    y = y + 1

100%|██████████| 91/91 [00:06<00:00, 13.17it/s]


In [18]:
with open('data.yaml', 'wt') as f:
    f.write('train: images/train \n')
    f.write('val: images/valid \n')
    f.write('test: images/test \n')
    f.write('dairy: images/dairy \n')
    f.write('paste: images/paste \n')
    f.write('support: supports \n')
    f.write(' \n')
    f.write('nc: '+str(i)+'\n')

In [19]:
!rm -rf dairy

In [20]:
!rm -rf grozi

In [21]:
!rm -rf paste

In [ ]:
# !rm grozi.zip

In [ ]:
# !rm retail_test_sets.zip